In [ ]:
%pip install -U -q google-genai python-dotenv

# Procesamiento de Lenguaje Natural con Google Gemini API

Este script demuestra diversas aplicaciones de Procesamiento de Lenguaje Natural (PLN) utilizando el modelo Gemini de Google a través de la librería `google-generativeai`.

In [ ]:
import os
from dotenv import load_dotenv

# Cargar variables de entorno desde .env
load_dotenv()

# Intentar detectar Colab
IN_COLAB = False
try:
    import google.colab
    from google.colab import userdata
    IN_COLAB = True
except Exception:
    IN_COLAB = False

GOOGLE_API_KEY = None
if IN_COLAB:
    # En Colab el usuario puede guardar secretos y recuperarlos con userdata.get
    try:
        GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    except Exception:
        GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
else:
    # En entorno local, usar dotenv para cargar desde .env
    GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

if not GOOGLE_API_KEY:
    raise ValueError('GOOGLE_API_KEY no encontrada. Coloca tu clave en .env o en Colab secrets con la key GOOGLE_API_KEY')

print(f"✅ GOOGLE key cargada. Entorno: {'Colab' if IN_COLAB else 'Local'}")

In [ ]:
from google import genai
from google.genai import types

cliente = genai.Client(api_key=GOOGLE_API_KEY)

In [ ]:
# Definición del texto de entrada
# Texto base que utilizaremos para todos los ejemplos.

text_to_process = """Estimado Amazon, la semana pasada pedí una figura de acción de Optimus Prime
en su tienda en línea en Alemania. Desafortunadamente, cuando abrí el paquete,
descubrí con horror que me habían enviado una figura de acción de Megatron
en su lugar. Como enemigo de toda la vida de los Decepticons, espero que pueda
entender mi dilema. Para resolver el problema, exijo un cambio de Megatron por
la figura de Optimus Prime que pedí. Adjunto copias de mis registros relativos
a esta compra. Espero tener noticias suyas pronto. Atentamente, Bumblebee."""

print("\nTexto de entrada definido.")

In [ ]:
MODEL_ID = "gemini-2.0-flash" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-flash-preview-05-20","gemini-2.5-pro-preview-05-06"] {"allow-input":true, isTemplate: true}

## 1. Sumarizacion

In [ ]:
pregunta = f"""Sumariza el siguiente texto en dos oraciones de rapida lectura

Texto: {text_to_process}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[pregunta] # Pasa la pregunta como contenido
)
print(respuesta.text)

##2. Clasificación de Sentimiento

In [ ]:
pregunta = f"""Clasificá el siguiente texto como positivo, negativo o neutral y explicá por qué:

Texto: {text_to_process}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[pregunta] # Pasa la pregunta como contenido
)
print(respuesta.text)

##3. Reconocimiento de Entidades Nombradas (NER)

In [ ]:
prompt = f"""Extraé todas las entidades nombradas del siguiente texto (personas, organizaciones, lugares, objetos) y clasificálas:

Texto: {text_to_process}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[prompt] # Pasa la pregunta como contenido
)
print(respuesta.text)

##4. Respuesta a preguntas (Question Answering)

In [ ]:
pregunta = "¿Qué producto recibió el cliente?"
contexto = text_to_process

prompt = f"""Respondé la siguiente pregunta basada en el texto:

Texto: {contexto}
Pregunta: {pregunta}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[contexto, pregunta] # Pasa la pregunta como contenido
)
print(respuesta.text)

##5. Resumen automático

In [ ]:
prompt = f"""Resumí el siguiente texto en no más de 3 líneas:

Texto: {text_to_process}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[prompt] # Pasa la pregunta como contenido
)
print(respuesta.text)

##6. Traducción (Español a Inglés)

In [ ]:
prompt = f"""Traducí al inglés este texto:

Texto: {text_to_process}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[prompt] # Pasa la pregunta como contenido
)
print(respuesta.text)

##7. Generación de respuesta (como atención al cliente)

In [ ]:
respuesta_inicial = "Estimado cliente, lamentamos mucho lo ocurrido con su pedido. "

prompt = f"""{text_to_process}

Redactá una respuesta del servicio de atención al cliente que comience así:

"{respuesta_inicial}"

Cuya extension no supere las 4 lineas.
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[prompt] # Pasa la pregunta como contenido
)
print(respuesta.text)

##8. Clasificación Zero-Shot (sin entrenamiento previo)

In [ ]:
etiquetas = ["queja", "elogio", "consulta", "pedido", "agradecimiento"]

prompt = f"""Clasificá el siguiente texto en una de estas categorías: {', '.join(etiquetas)}. Justificá tu elección.

Texto: {text_to_process}
"""

respuesta = cliente.models.generate_content(
    model=MODEL_ID,
    contents=[prompt] # Pasa la pregunta como contenido
)
print(respuesta.text)

## EJERCICIO

Escribí un texto corto sobre una experiencia personal en un transporte público en Buenos Aires.

Luego, generá:

- Un resumen.
- Una clasificación de sentimiento.
- Una lista de entidades nombradas.